### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220714_scenario_projections.ipynb")

2022/07/14 22:24:25 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [ ]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220628_july_gov_briefing/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

### Create and run scenarios from Feb 2022 to present

In [3]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220628_july_gov_briefing/scenario_params.json"))

model_args = {
    'base_spec_id': 3214 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for beta_mult in [1.0, 0.9, 1.1]:
    for hosp_frac in [0.5, 0.7, 0.9]: # 50%, 70% , or 90% for-COVID
        for school_contact_increase in [1.0, 1.1, 1.2]:
            for immune_escape_mult in [1.0, 0.9, 1.1]:
                hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
                beta_param_adjustment = [{"param": "betta", "attrs": {"variant": "ba45"}, "mults": {"2020-01-01": beta_mult}, "desc": "sensitivity analysis for beta"}]
                school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},"desc": "After August 15th, beta increases by a factor of the school contact increase"}]
                lt5_vacc_adjust = [{"param": "immunity", "attrs": {'age': '0-19', 'vacc': 'shot1'}, "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5, "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5}}]
                immune_mult = [{"param": "immune_escape", "from_attrs":{"variant": ["omicron", "ba2", "ba2121"]}, "to_attrs": {"variant": ["ba45"]}, "mults": {"2020-01-01": immune_escape_mult},  "desc": "sensitivity analysis for immune escape"}]
                scenario_model_args.append({'params_defs': scenario_params + beta_param_adjustment + school_contact + lt5_vacc_adjust + immune_mult,
                                            'hosp_reporting_frac': hrf,
                                            'tags': {'beta_mult': beta_mult, 'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase, 'immune_escape_mult': immune_escape_mult}})
            
            

In [4]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/07/14 22:24:39 INFO|============================================================
2022/07/14 22:24:39 INFO|============================================================
2022/07/14 22:24:39 INFO|============================================================
2022/07/14 22:24:39 INFO|============================================================
2022/07/14 22:24:39 INFO|============================================================
2022/07/14 22:24:39 INFO|============================================================
2022/07/14 22:24:42 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.1} Retrieving vaccinations data
2022/07/14 22:24:42 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9} Retrieving vaccinations data
2022/07/14 22:24:42 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.1} Retrieving vaccinations dat

2022/07/14 22:25:58 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.0, 'fit_batch': '0'}: Transmission control fit 1/2 completed in 13.813491916000004 seconds: {767: {'co': 0.77091913141751}, 781: {'co': 0.8174472129562174}, 795: {'co': 0.7964212414253298}, 809: {'co': 0.8048850966151335}, 823: {'co': 0.8060124198580787}, 837: {'co': 0.7950010193701275}}
2022/07/14 22:25:58 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.0, 'fit_batch': '0'}: Running forward sim
2022/07/14 22:25:58 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.9, 'fit_batch': '0'}: Transmission control fit 1/2 completed in 13.862662291999996 seconds: {767: {'co': 0.7709187944966553}, 781: {'co': 0.8174467718047981}, 795: {'co': 0.7964215974597803}, 809: {'co': 0.8048832898582645}, 823: {'co': 0.8059102768850696}, 837: {'co': 0.794506125320206

2022/07/14 22:26:15 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0, 'fit_batch': '1'}: Transmission control fit 2/2 completed in 15.710531458000006 seconds: {795: {'co': 0.7988855019407686}, 809: {'co': 0.8149220836466635}, 823: {'co': 0.8165341803302889}, 837: {'co': 0.8165374066963516}, 851: {'co': 0.8147010701522932}, 865: {'co': 0.8337964314134566}}
2022/07/14 22:26:15 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0, 'fit_batch': '1'}: Running forward sim
2022/07/14 22:26:16 INFO|============================================================
2022/07/14 22:26:16 INFO|============================================================
2022/07/14 22:26:16 INFO|============================================================
2022/07/14 22:26:16 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9, 'fit_batch': '1'}: Trans

2022/07/14 22:26:20 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.1} Retrieving vaccinations data
2022/07/14 22:26:20 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 0.9} Prepping Model
2022/07/14 22:26:20 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 0.9} Prepping Model
2022/07/14 22:26:20 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.1} Prepping Model
2022/07/14 22:26:20 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.1} Prepping Model
2022/07/14 22:26:20 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.0} Prepping Model
2022/07/14 22:26:20 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'immune

2022/07/14 22:27:23 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.1} Model solved in 2.561909958000001 seconds.
2022/07/14 22:27:23 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.1} Will fit 2 times
2022/07/14 22:27:24 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0} Model prepped for fitting in 58.336457042000006 seconds.
2022/07/14 22:27:24 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0} Solving Model ODE
2022/07/14 22:27:27 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0} Model solved in 2.48433 seconds.
2022/07/14 22:27:27 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0} Will fit 2 times
2022/07/14 22:27:34 INFO|--------|{

...

2022/07/14 22:49:15 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.9, 'fit_batch': '1'}: Running forward sim
2022/07/14 22:49:16 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.9, 'fit_batch': '1', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5313747878505213}, 42: {'co': 9.166775206693499e-14}, 56: {'co': 0.7004806299042426}, 70: {'co': 0.8356320023671157}, 84: {'co': 0.8298063033343771}, 98: {'co': 0.8311909839169287}, 112: {'co': 0.8778137464237421}, 126: {'co': 0.8241103755501108}, 140: {'co': 0.7606632825445033}, 154: {'co': 0.6711880168053298}, 168: {'co': 0.8430307682247831}, 182: {'co': 0.7826561801378445}, 196: {'co': 0.8191830543380944}, 210: {'co': 0.7726034259026388}, 224: {'co': 0.7713676622018327}, 238: {'co': 0.7185596188624429}, 252: {'co': 0.7222209085605668}, 266: {'co': 0.684882273037046

2022/07/14 22:49:45 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.9, 'fit_batch': '1', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5313747878505213}, 42: {'co': 9.166775206693499e-14}, 56: {'co': 0.7004806299042426}, 70: {'co': 0.8356320023671157}, 84: {'co': 0.8298063033343771}, 98: {'co': 0.8311909839169287}, 112: {'co': 0.8778137464237421}, 126: {'co': 0.8241103755501108}, 140: {'co': 0.7606632825445033}, 154: {'co': 0.6711880168053298}, 168: {'co': 0.8430307682247831}, 182: {'co': 0.7826561801378445}, 196: {'co': 0.8191830543380944}, 210: {'co': 0.7726034259026388}, 224: {'co': 0.7713676622018327}, 238: {'co': 0.7185596188624429}, 252: {'co': 0.7222209085605668}, 266: {'co': 0.6848822730370463}, 280: {'co': 0.7199443312231306}, 294: {'co': 0.7575590209953923}, 308: {'co': 0.8226615649781124}, 322: {'co': 0.8113711381077103}, 336: {'co': 0.7825940193755743}, 3

2022/07/14 22:51:58 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.0} Prepping Model
2022/07/14 22:52:57 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.1} Model prepped for fitting in 67.15371275000007 seconds.
2022/07/14 22:52:57 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.1} Solving Model ODE
2022/07/14 22:52:59 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.1} Model solved in 2.384604667000076 seconds.
2022/07/14 22:52:59 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.1} Will fit 2 times
2022/07/14 22:53:02 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.0} Model prepped for fitting in 64.1077533749999 seconds.
2022/07/14 22:5

2022/07/14 22:53:47 INFO|============================================================
2022/07/14 22:53:49 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9} Retrieving vaccinations data
2022/07/14 22:53:50 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9} Constructing vaccination projections
2022/07/14 22:53:50 INFO|-----------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9} Retrieving hospitalizations data
2022/07/14 22:53:51 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9} Prepping Model
2022/07/14 22:53:51 INFO|---------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9} Prepping Model
2022/07/14 22:54:41 INFO|--------|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'immune_escape_mult': 

2022/07/14 23:03:54 INFO|-----------------------|spec_ids: 3377,3378,3379,3380,3381,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3410,3411,3412,3413,3414,3415,3416,3417,3418,3419,3420,3421,3422,3423,3424,3425,3426,3427,3428,3429,3430,3431,3432,3433,3434,3435,3436,3437,3438,3439,3440,3441,3442,3443,3444,3445,3446,3447,3448,3449,3450,3451,3452,3453,3454,3455,3456,3457


### Run the report for each fit model

In [5]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

2022/07/14 23:12:34 INFO|============================================================
2022/07/14 23:12:34 INFO|--------|Solving model
2022/07/14 23:12:36 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:12:42 INFO|============================================================
2022/07/14 23:12:42 INFO|--------|Solving model
2022/07/14 23:12:44 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:12:52 INFO|============================================================
2022/07/14 23:12:52 INFO|--------|Solving model
2022/07/14 23:12:55 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:13:04 INFO|==================

2022/07/14 23:19:50 INFO|============================================================
2022/07/14 23:19:50 INFO|--------|Solving model
2022/07/14 23:19:53 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:20:09 INFO|============================================================
2022/07/14 23:20:09 INFO|--------|Solving model
2022/07/14 23:20:12 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:20:35 INFO|============================================================
2022/07/14 23:20:35 INFO|--------|Solving model
2022/07/14 23:20:38 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:21:03 INFO|==================

2022/07/14 23:31:29 INFO|============================================================
2022/07/14 23:31:29 INFO|--------|Solving model
2022/07/14 23:31:32 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:31:53 INFO|============================================================
2022/07/14 23:31:53 INFO|--------|Solving model
2022/07/14 23:31:57 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:32:01 INFO|============================================================
2022/07/14 23:32:01 INFO|--------|Solving model
2022/07/14 23:32:04 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:32:28 INFO|==================

2022/07/14 23:43:45 INFO|============================================================
2022/07/14 23:43:45 INFO|--------|Solving model
2022/07/14 23:43:48 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:44:42 INFO|============================================================
2022/07/14 23:44:42 INFO|--------|Solving model
2022/07/14 23:44:45 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/07/14 23:45:07 INFO|============================================================
2022/07/14 23:45:07 INFO|--------|Solving model
2022/07/14 23:45:10 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [6]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/07/15 08:38:59 INFO|Projecting
2022/07/15 08:38:59 INFO|
2022/07/15 08:39:08 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.0, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:39:10 INFO|
2022/07/15 08:39:19 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 0.9, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:39:20 INFO|
2022/07/15 08:39:29 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_mult': 1.1, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:39:30 INFO|
2022/07/15 08:43:33 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.0, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:43:35 INFO|
2022/07/15 08:43:44 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.5, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.

2022/07/15 08:52:45 INFO|
2022/07/15 08:52:54 INFO|{'beta_mult': 0.9, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 0.9, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:52:55 INFO|
2022/07/15 08:53:04 INFO|{'beta_mult': 0.9, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'immune_escape_mult': 1.1, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:53:05 INFO|
2022/07/15 08:53:14 INFO|{'beta_mult': 0.9, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.0, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:53:16 INFO|
2022/07/15 08:54:51 INFO|{'beta_mult': 0.9, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 0.9, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 08:54:53 INFO|
2022/07/15 08:55:02 INFO|{'beta_mult': 0.9, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.1, 'fit_batch': '1', 'run_type': 'fi

2022/07/15 09:04:46 INFO|
2022/07/15 09:04:58 INFO|{'beta_mult': 1.1, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'immune_escape_mult': 1.1, 'fit_batch': '1', 'run_type': 'fit'}: Running forward sim
2022/07/15 09:04:59 INFO|Running reports
